In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [25]:
#Import the dataset
data_train=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/DataTest/LSTM1/Train_56000.csv')
data_test=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/DataTest/LSTM1/Test_14000.csv')

X_train=data_train.iloc[:56000,[1,3,4]].values
Y_train=data_train.iloc[:56000,0].values

X_test=data_test.iloc[:14000,[1,3,4]].values
Y_test=data_test.iloc[:14000,0].values


In [26]:
X_train, Y_train, X_test, Y_test = list(map(lambda x: np.array(x, dtype=np.float64), [X_train, Y_train, X_test, Y_test]))

In [27]:
data_train.head()

,LABEL,ip.id,tcp.seq_raw,frame.len,frame.cap_len,dns.qry.name,http.user_agent
0,0,18555,0,60,42,0.0,0
1,1,30673,1082820181,266,266,0.0,0
2,0,27819,740183112,54,54,0.0,0
3,0,28353,1099952435,93,66,0.0,0
4,0,6571,0,60,42,0.0,0


In [ ]:
X_train[0]

In [45]:
max_words = 65526
max_len = 3
# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
# sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix = X_train
Y_train = Y_train.reshape(-1, 1)

In [46]:
Y_train.shape

(56000, 1)

In [47]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    # layer = Merge([max_len, context_model], mode="dot", dot_axes=0)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [48]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 3)]               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 3, 50)             3276800   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [49]:
model.fit(sequences_matrix,Y_train,batch_size=1024,epochs=10,
          validation_split=0.2)

Epoch 1/10
44/44 [==============================] - 3s 27ms/step - loss: 0.3074 - accuracy: 0.9169 - val_loss: 0.0656 - val_accuracy: 0.9846
Epoch 2/10
44/44 [==============================] - 1s 14ms/step - loss: 0.0323 - accuracy: 0.9917 - val_loss: 0.0390 - val_accuracy: 0.9896
Epoch 3/10
44/44 [==============================] - 1s 14ms/step - loss: 0.0144 - accuracy: 0.9956 - val_loss: 0.0523 - val_accuracy: 0.9803
Epoch 4/10
44/44 [==============================] - 1s 15ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.0513 - val_accuracy: 0.9861
Epoch 5/10
44/44 [==============================] - 1s 15ms/step - loss: 0.0053 - accuracy: 0.9984 - val_loss: 0.0497 - val_accuracy: 0.9862
Epoch 6/10
44/44 [==============================] - 1s 15ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 0.0458 - val_accuracy: 0.9886
Epoch 7/10
44/44 [==============================] - 1s 15ms/step - loss: 0.0034 - accuracy: 0.9990 - val_loss: 0.0478 - val_accuracy: 0.9862
Epoch 8/10
44

In [51]:
model.evaluate(X_test, Y_test)

438/438 [==============================] - 1s 3ms/step - loss: 0.0462 - accuracy: 0.9863


[0.04616327956318855, 0.9862856864929199]

In [54]:
model.predict(np.array([X_test[4]]))

array([[0.9999355]], dtype=float32)

In [55]:
Y_test[4]

1.0